
## Calling tools with any models.

https://python.langchain.com/docs/use_cases/tool_use/prompting/

In [1]:
from langchain.agents import tool
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()




In [2]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [3]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [4]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

# create a prompt

In [5]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
Please also give some explanation of the tool you are using with key 'explanation'.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nmultiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.\nPlease also give some explanation of the tool you are using with key 'explanation'.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [8]:
## Adding an output parser to the tool
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
model = ChatOpenAI(
    model_name="qwen32b-distill-r1",
    openai_api_key="EMPTY",
    openai_api_base="http://10.4.119.108:8000/v1",
    # http_client=http_client
)

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply',
 'arguments': {'first_int': 13, 'second_int': 4},
 'explanation': "The multiply tool is used to find the product of two integers. Here, we're calculating 13 multiplied by 4, which equals 52."}

In [10]:
## calling tools
from operator import itemgetter

chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "what's thirteen times 4"})

52

In [11]:
# 选择多个工具
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [12]:
# 增加了三个工具
tools = [add, exponentiate, multiply]

# 根据大模型选择来调用工具，和官方不一样，增加了可以选择多个工具
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    for selected in model_output:
        selected["output"] = tool_map[selected["name"]].invoke(selected["arguments"])
    return model_output


In [13]:
## 构建 prompt
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
Give reasons for your choice of tool with the key 'explanation', use chinese for its value.
There could be more than one tool to use, so you should return a list of JSON blobs with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | model | JsonOutputParser() | tool_chain

# 同时计算两个没问题，但是如果要连续计算就不行了
chain.invoke({"input": "what's 4 power 20, and then times 4, and then add 5"})



ValidationError: 1 validation error for multiplySchema
first_int
  value is not a valid integer (type=type_error.integer)

In [14]:
# 将工具运行结果及 input 都输出出来
from langchain_core.runnables import RunnablePassthrough


# 下面是官方例子
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

## 构建 prompt
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

# chain = prompt | model | JsonOutputParser() | tool_chain

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)
)
chain.invoke({"input": "what's 3 plus 1132, and 5 powers 3"})

{'name': 'add',
 'arguments': {'first_int': 3, 'second_int': 1132},
 'output': 1135}

In [14]:
# 尝试增加多轮对话
# 将工具运行结果及 input 都输出出来
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder



# 下面是官方例子
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

## 构建 prompt
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("user", "{input}")
    ]
)

# chain = prompt | model | JsonOutputParser() | tool_chain

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)
)
question = "what's 3 plus 1132"
result = await chain.ainvoke({"input": question})
result

{'name': 'add',
 'arguments': {'first_int': 3, 'second_int': 1132},
 'output': 1135}

In [33]:
from langchain_core.messages import HumanMessage, AIMessage

# 开启第二轮对话，注意变量名称需要和 prompt 中的占位符一致
chat_history = []
chat_history.extend([HumanMessage(content=question), AIMessage(content=result["output"])])
question_new = "Then double this value?"
print(chat_history)
print(chain.get_prompts())
new_result = chain.invoke({"input": question_new, "chat_history": chat_history})
new_result

[HumanMessage(content="what's 3 plus 1132"), AIMessage(content='1135')]
[ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nadd: add(first_int: int, second_int: int) -> int - Add two integers.\nexponentiate: exponentiate(base: int, exponent: int) -> int - Exponentiate the base to the exponent power.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the nam

{'name': 'multiply',
 'arguments': {'first_int': 1135, 'second_int': 2},
 'output': 2270}